In [2]:
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.2. Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando num ponto para destacá-lo. Usando o dataset `cars`, vamos começar com um gráfico de dispersão da potência (em cavalos) por milhas por galão, com uma codificação de cor para o número de cilindros no motor do carro.

Além disso, vamos criar uma instância de seleção chamando `alt.selection_single()`, indicando que queremos uma seleção definida por um _único valor_. Por padrão, a seleção usa um clique de mouse para determinar o valor selecionado. Para registrar uma seleção com um gráfico, devemos adicionar usando o método `.add_selection()`.

Uma vez que nossa seleção tenha sido definida, podemos usá-la como parâmetro para _codificações condicionais_, que aplicam uma codificação diferente dependendo se o registro do dado está dentro ou fora da seleção. Por exemplo, considere o seguinte código:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

Essa definição de codificação afirma que os pontos do dado contidos em `selection` devem ser coloridos de acordo com o campo `Cylinder`, enquanto pontos do dado não selecionados devem usar um `grey` (_cinza_) padrão. Uma seleção vazia inclui _todos_ os pontos do dado e, então, inicialmente todos os pontos serão coloridos.

_Tente clicar em diferentes pontos no gráfico abaixo. O que acontece? (Clique no fundo para limpar o estado da seleção e retornar a uma seleção "vazia".)_

In [13]:
selection = alt.selection_single()
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x= alt.X('Horsepower:Q', title = 'Potência (em cavalos)'),
    y= alt.Y('Miles_per_Gallon:Q', title = 'Milhas por galão'),
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

/tmp/ipykernel_8892/1833167720.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single()
/tmp/ipykernel_8892/1833167720.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(cars).mark_circle().add_selection(


alt.Chart(...)

Claro, destacar pontos do dado individuais um a um não é particularmente divertido! Como veremos, seleções de valor único fornecem um alicerce útil para construir interações mais poderosas. Ademais, seleções de valor único são apenas uma dos três tipos de seleção fornecidas pelo Altair:

- `selection_single` - selecione um único valor discreto, por padrão em eventos de clique.
- `selection_multi` - selecione vários valores discretos. O primeiro valor é selecionado clicando com o mouse e valores adicionais são inseridos usando Shift+Click. 
- `selection_interval` - selecione uma faixa contínua de valores, iniciados arrastando o mouse enquanto clicado.

Vamos comparar cada uma desses tipos de seleção lado a lado. Para manter nosso código arrumado, vamos primeiro definir uma função (`plot`) que gera especificações dos gráficos de dispersão como o acima. Podemos passar uma seleção para a função `plot` para aplicá-lo ao gráfico:

In [14]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x= alt.X('Horsepower:Q', title = 'Potência (em cavalos)'),
        y= alt.Y('Miles_per_Gallon:Q', title = 'Milhas por galão'),
        color=alt.condition(selection, alt.Color('Cylinders:O', title = 'Cilindros'), alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar nossa função `plot` para criar três variantes do gráfico, um para cada tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) suporta a interação Shift+Click para inserir vários pontos na seleção. O terceiro gráfico (`interval`) gera uma região selecionada (ou _brush_) a partir do arrasto do mouse clicado. Uma vez criada, você pode mover a área varrida pelo gráfico para selecionar diferentes pontos, ou rolar quando o cursor está dentro da área varrida para aumentar ou diminuir o tamanho da área.

_Tente interagir com os gráficos abaixo!_

In [15]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

/tmp/ipykernel_8892/2299552586.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_single()).properties(title='Single (Click)'),
/tmp/ipykernel_8892/3048183788.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  return alt.Chart(cars).mark_circle().add_selection(
/tmp/ipykernel_8892/2299552586.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),


alt.HConcatChart(...)

Os exemplos acima usam interações padrão (Clicar, Shift+Click, Arrastar) para cada tipo de seleção. Podemos costumizar mais as interações fornecendo especificações do evento de entrada usando a [sintaxe de seleção de evento do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para acionar com eventos de `pairar do mouse` ao invés de eventos de `click`

_Segure a tecla Shift no segundo gráfico para "pintar" com os dados!_

In [16]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Pairar do Mouse)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Pairar do Mouse)')
)

/tmp/ipykernel_8892/4232158512.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Pairar do Mouse)'),
/tmp/ipykernel_8892/3048183788.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  return alt.Chart(cars).mark_circle().add_selection(
/tmp/ipykernel_8892/4232158512.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Pairar do Mouse)')


alt.HConcatChart(...)

Agora que cobrimos o básico das seleções do Altair, vamos fazer um tour pelas diversas técnicas de interação que elas permitem!